This code is for managing the API result

Set the environment variable GOOGLE_APPLICATION_CREDENTIALS to the file path of the JSON file that contains your service account key.

In [4]:
import argparse
import io
import re

from google.cloud import storage
from google.cloud import vision
from google.protobuf import json_format


In [5]:
# [START vision_web_detection]
def detect_web(path):
    """Detects web annotations given an image."""
    client = vision.ImageAnnotatorClient()

    # [START vision_python_migration_web_detection]
    with io.open(path, 'rb') as image_file:
        content = image_file.read()

    image = vision.types.Image(content=content)

    response = client.web_detection(image=image)
    annotations = response.web_detection
    print(type(annotations))
    if annotations.best_guess_labels:
        for label in annotations.best_guess_labels:
            print('\nBest guess label: {}'.format(label.label))

    if annotations.pages_with_matching_images:
        print('\n{} Pages with matching images found:'.format(
            len(annotations.pages_with_matching_images)))

        for page in annotations.pages_with_matching_images:
            print('\n\tPage url   : {}'.format(page.url))

            if page.full_matching_images:
                print('\t{} Full Matches found: '.format(
                       len(page.full_matching_images)))

                for image in page.full_matching_images:
                    print('\t\tImage url  : {}'.format(image.url))

            if page.partial_matching_images:
                print('\t{} Partial Matches found: '.format(
                       len(page.partial_matching_images)))

                for image in page.partial_matching_images:
                    print('\t\tImage url  : {}'.format(image.url))

    if annotations.web_entities:
        print('\n{} Web entities found: '.format(
            len(annotations.web_entities)))

        for entity in annotations.web_entities:
            print('\n\tScore      : {}'.format(entity.score))
            print(u'\tDescription: {}'.format(entity.description))

    if annotations.visually_similar_images:
        print('\n{} visually similar images found:\n'.format(
            len(annotations.visually_similar_images)))

        for image in annotations.visually_similar_images:
            print('\tImage url    : {}'.format(image.url))
            
    return annotations
    # [END vision_python_migration_web_detection]
# [END vision_web_detection]

In [21]:
def localize_objects(path):
    """Localize objects in the local image.

    Args:
    path: The path to the local file.
    """
    from google.cloud import vision
    client = vision.ImageAnnotatorClient()

    with open(path, 'rb') as image_file:
        content = image_file.read()
    image = vision.types.Image(content=content)

    objects = client.object_localization(
        image=image).localized_object_annotations

    print('Number of objects found: {}'.format(len(objects)))
    objs = dict()
    for object_ in objects:
        print('\n{} (confidence: {})'.format(object_.name, object_.score))
        print('Normalized bounding polygon vertices: ')
        objs[object_.name.lower()] = object_.score
        for vertex in object_.bounding_poly.normalized_vertices:
            print(' - ({}, {})'.format(vertex.x, vertex.y))
    return objs

In [22]:
def detect_labels(path):
    """Detects labels in the file."""
    from google.cloud import vision
    client = vision.ImageAnnotatorClient()

    with io.open(path, 'rb') as image_file:
        content = image_file.read()

    image = vision.types.Image(content=content)

    response = client.label_detection(image=image)
    labels = response.label_annotations
    print('Labels:')
    descriptions = dict()
    for label in labels:
        print(label.description)
        descriptions[label.description.lower()]=label.score
    return descriptions

In [23]:
labels = detect_labels('C:/Users/ADMIIN/Desktop/fake.jpg')
obj = localize_objects('C:/Users/ADMIIN/Desktop/fake.jpg')
web = detect_web('C:/Users/ADMIIN/Desktop/fake.jpg')

Labels:
eyewear
sunglasses
vision care
glasses
product
goggles
product
plastic
personal protective equipment
glass
Number of objects found: 4

Sunglasses (confidence: 0.9642857313156128)
Normalized bounding polygon vertices: 
 - (0.11052388697862625, 0.21080206334590912)
 - (0.8838289976119995, 0.21080206334590912)
 - (0.8838289976119995, 0.4729626178741455)
 - (0.11052388697862625, 0.4729626178741455)

Sunglasses (confidence: 0.8793388605117798)
Normalized bounding polygon vertices: 
 - (0.1333974003791809, 0.6469928026199341)
 - (0.6004850268363953, 0.6469928026199341)
 - (0.6004850268363953, 0.9160661101341248)
 - (0.1333974003791809, 0.9160661101341248)

Glasses (confidence: 0.7330300807952881)
Normalized bounding polygon vertices: 
 - (0.11052388697862625, 0.21080206334590912)
 - (0.8838289976119995, 0.21080206334590912)
 - (0.8838289976119995, 0.4729626178741455)
 - (0.11052388697862625, 0.4729626178741455)

Goggles (confidence: 0.6685091257095337)
Normalized bounding polygon ver

In [24]:
labels

{'eyewear': 0.9938976168632507,
 'glass': 0.558018684387207,
 'glasses': 0.9103922247886658,
 'goggles': 0.8035570383071899,
 'personal protective equipment': 0.6138655543327332,
 'plastic': 0.6163051128387451,
 'product': 0.7874263525009155,
 'sunglasses': 0.9657864570617676,
 'vision care': 0.9368029236793518}

In [25]:
obj

{'glasses': 0.7330300807952881,
 'goggles': 0.6685091257095337,
 'sunglasses': 0.8793388605117798}

In [14]:
if 'sunglasses' in labels:
    print(labels['sunglasses'])

0.9657862782478333


In [20]:
if ('Sunglasses' in obj) or ('sunglasses' in obj) :
    print(obj['Sunglasses'] or obj['sunglasses'])

0.8793388605117798


In [60]:
"""annotations.best_guess_labels:
        #for label in annotations.best_guess_labels:
            print('\nBest guess label: {}'.format(label.label))

    if annotations.pages_with_matching_images:
        for page in annotations.pages_with_matching_images:
            print(.format(page.url))

            if page.full_matching_images:
                print('\t{} Full Matches found: '.format(
                       len(page.full_matching_images)))

                for image in page.full_matching_images:
                    print('\t\tImage url  : {}'.format(image.url))

            if page.partial_matching_images:
                print('\t{} Partial Matches found: '.format(
                       len(page.partial_matching_images)))

                for image in page.partial_matching_images:
                    print('\t\tImage url  : {}'.format(image.url))

    if annotations.web_entities:
        print('\n{} Web entities found: '.format(
            len(annotations.web_entities)))

        for entity in annotations.web_entities:
            print('\n\tScore      : {}'.format(entity.score))
            print(u'\tDescription: {}'.format(entity.description))

    if annotations.visually_similar_images:
        print('\n{} visually similar images found:\n'.format(
            len(annotations.visually_similar_images)))

        for image in annotations.visually_similar_images:
            print('\tImage url    : {}'.format(image.url))
            """
web

web_entities {
  entity_id: "/g/11dxcp3fzp"
  score: 0.8313999772071838
  description: "Ray-Ban Round Double Bridge"
}
web_entities {
  entity_id: "/m/02m7zw"
  score: 0.7394999861717224
  description: "Ray-Ban"
}
web_entities {
  entity_id: "/t/21bbnhtdrdx0"
  score: 0.7394999861717224
}
web_entities {
  entity_id: "/g/11cpg1np0k"
  score: 0.7276999950408936
  description: "Ray-Ban Round Metal"
}
web_entities {
  entity_id: "/g/11c2nxmcsh"
  score: 0.7071999907493591
  description: "Ray-Ban Aviator Flash"
}
web_entities {
  entity_id: "/g/11c5hlclxs"
  score: 0.7026000022888184
  description: "Ray-Ban Aviator Classic"
}
web_entities {
  entity_id: "/m/017ftj"
  score: 0.701200008392334
  description: "Sunglasses"
}
web_entities {
  entity_id: "/g/11g8ksb0p3"
  score: 0.7002000212669373
  description: "Ray-Ban Blaze Round"
}
web_entities {
  entity_id: "/g/11d_7ptj_k"
  score: 0.6772000193595886
  description: "Ray-Ban Blaze Clubmaster"
}
web_entities {
  entity_id: "/g/11cs24zpsz"
  s